In [1]:
# Import the libraries

import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import sklearn.metrics as skmet
import pickle


In [2]:
df = pd.read_csv('cancerdata.csv')
df.head()

,id,diagnosis,Sex,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,points_worst,symmetry_worst,dimension_worst
0,87139402,B,F,12.32,12.39,78.85,464.1,0.10280,0.06981,0.03987,...,13.50,15.64,86.97,549.1,0.1385,0.1266,0.12420,0.09391,0.2827,0.06771
1,8910251,B,M,10.60,18.95,69.28,346.4,0.09688,0.11470,0.06387,...,11.88,22.94,78.28,424.8,0.1213,0.2515,0.19160,0.07926,0.2940,0.07587
2,905520,B,M,11.04,16.83,70.92,373.2,0.10770,0.07804,0.03046,...,12.41,26.44,79.93,471.4,0.1369,0.1482,0.10670,0.07431,0.2998,0.07881
3,868871,B,F,11.28,13.39,73.00,384.8,0.11640,0.11360,0.04635,...,11.92,15.77,76.53,434.0,0.1367,0.1822,0.08669,0.08611,0.2102,0.06784
4,9012568,B,F,15.19,13.21,97.65,711.8,0.07963,0.06934,0.03393,...,16.20,15.73,104.50,819.1,0.1126,0.1737,0.13620,0.08178,0.2487,0.06766


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 569 non-null    int64  
 1   diagnosis          569 non-null    object 
 2   Sex                569 non-null    object 
 3   radius_mean        569 non-null    float64
 4   texture_mean       569 non-null    float64
 5   perimeter_mean     569 non-null    float64
 6   area_mean          569 non-null    float64
 7   smoothness_mean    569 non-null    float64
 8   compactness_mean   569 non-null    float64
 9   concavity_mean     567 non-null    float64
 10  points_mean        568 non-null    float64
 11  symmetry_mean      568 non-null    float64
 12  dimension_mean     569 non-null    float64
 13  radius_se          569 non-null    float64
 14  texture_se         569 non-null    float64
 15  perimeter_se       569 non-null    float64
 16  area_se            569 non

In [4]:
# Data Preprocessing & EDA
# converting B to Benign and M to Malignant 
df['diagnosis'] = np.where(df['diagnosis'] == 'B', 'Benign', df['diagnosis'])
df['diagnosis'] = np.where(df['diagnosis'] == 'M', 'Malignant', df['diagnosis'])

In [5]:
df.drop(['id'], axis = 1, inplace = True) # Excluding id column

In [6]:
df.describe()


,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,points_mean,symmetry_mean,dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,points_worst,symmetry_worst,dimension_worst
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,567.000000,568.000000,568.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088911,0.048846,0.181161,0.062798,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079837,0.038797,0.027438,0.007060,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029520,0.020310,0.161900,0.057700,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033455,0.179200,0.061540,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.131300,0.073730,0.195700,0.066120,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


In [7]:
# Seperating input and output variables 
df_X = pd.DataFrame(df.iloc[:, 1:])
df_y = pd.DataFrame(df.iloc[:, 0])


In [8]:
# All numeric features
numeric_features = df_X.select_dtypes(exclude = ['object']).columns

numeric_features


Index(['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
       'smoothness_mean', 'compactness_mean', 'concavity_mean', 'points_mean',
       'symmetry_mean', 'dimension_mean', 'radius_se', 'texture_se',
       'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se',
       'concavity_se', 'points_se', 'symmetry_se', 'dimension_se',
       'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst',
       'smoothness_worst', 'compactness_worst', 'concavity_worst',
       'points_worst', 'symmetry_worst', 'dimension_worst'],
      dtype='object')

In [9]:
# Imputation strategy for numeric columns
num_pipeline = Pipeline([('impute', SimpleImputer(strategy = 'mean'))])



In [10]:
# All categorical features
categorical_features = df_X.select_dtypes(include = ['object']).columns


In [11]:
from sklearn_pandas import DataFrameMapper

# DataFrameMapper is used to map the given Attribute
# Encoding categorical to numeric variable
categ_pipeline = Pipeline([('label', DataFrameMapper([(categorical_features,
                                                       OneHotEncoder(drop = 'first'))]))])


In [12]:
# Using ColumnTransfer to transform the columns of an array or pandas DataFrame. This estimator allows different columns or column subsets of the input to be transformed separately and the features generated by each transformer will be concatenated to form a single feature space.
preprocess_pipeline = ColumnTransformer([('categorical', categ_pipeline, categorical_features), 
                                       ('numerical', num_pipeline, numeric_features)])


In [13]:
processed = preprocess_pipeline.fit(df_X)  # Pass the raw data through pipeline

processed


ColumnTransformer(transformers=[('categorical',
                                 Pipeline(steps=[('label',
                                                  DataFrameMapper(drop_cols=[],
                                                                  features=[(Index(['Sex'], dtype='object'),
                                                                             OneHotEncoder(drop='first'))]))]),
                                 Index(['Sex'], dtype='object')),
                                ('numerical',
                                 Pipeline(steps=[('impute', SimpleImputer())]),
                                 Index(['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
       'smoothness_mean', 'c...
       'symmetry_mean', 'dimension_mean', 'radius_se', 'texture_se',
       'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se',
       'concavity_se', 'points_se', 'symmetry_se', 'dimension_se',
       'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst',
       'smoothness_worst', 'compactness_worst', 'concavity_worst',
       'points_worst', 'symmetry_worst', 'dimension_worst'],
      dtype='object'))])

In [14]:
# Save the defined pipeline
import joblib
joblib.dump(processed, 'processed1')


['processed1']

In [15]:
# Transform the original data using the pipeline defined above
clean = pd.DataFrame(processed.transform(df_X), columns = df_X.columns)  # Clean and processed data for Clustering

clean.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Sex                569 non-null    float64
 1   radius_mean        569 non-null    float64
 2   texture_mean       569 non-null    float64
 3   perimeter_mean     569 non-null    float64
 4   area_mean          569 non-null    float64
 5   smoothness_mean    569 non-null    float64
 6   compactness_mean   569 non-null    float64
 7   concavity_mean     569 non-null    float64
 8   points_mean        569 non-null    float64
 9   symmetry_mean      569 non-null    float64
 10  dimension_mean     569 non-null    float64
 11  radius_se          569 non-null    float64
 12  texture_se         569 non-null    float64
 13  perimeter_se       569 non-null    float64
 14  area_se            569 non-null    float64
 15  smoothness_se      569 non-null    float64
 16  compactness_se     569 non

In [16]:
# new_features = cancerclean.select_dtypes(exclude = ['object']).columns 
# new_features


In [17]:
# Define scaling pipeline
scale_pipeline = Pipeline([('scale', MinMaxScaler())])



In [18]:
preprocess_pipeline2 = ColumnTransformer([('scale', scale_pipeline, clean.columns)]) 

processed2 = preprocess_pipeline2.fit(clean)
processed2


ColumnTransformer(transformers=[('scale',
                                 Pipeline(steps=[('scale', MinMaxScaler())]),
                                 Index(['Sex', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
       'smoothness_mean', 'compactness_mean', 'concavity_mean', 'points_mean',
       'symmetry_mean', 'dimension_mean', 'radius_se', 'texture_se',
       'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se',
       'concavity_se', 'points_se', 'symmetry_se', 'dimension_se',
       'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst',
       'smoothness_worst', 'compactness_worst', 'concavity_worst',
       'points_worst', 'symmetry_worst', 'dimension_worst'],
      dtype='object'))])

In [19]:
# Save the Scaling pipeline
joblib.dump(processed2, 'processed2')


['processed2']

In [20]:
# Normalized data frame (considering the numerical part of data)

clean_n = pd.DataFrame(processed2.transform(clean), columns = clean.columns)


In [21]:
clean_n.describe()

,Sex,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,points_worst,symmetry_worst,dimension_worst
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,0.467487,0.338222,0.323965,0.332935,0.216920,0.394785,0.260601,0.208320,0.242772,0.379601,...,0.296663,0.363998,0.283138,0.170906,0.404138,0.220212,0.217403,0.393836,0.263307,0.189596
std,0.499381,0.166787,0.145453,0.167915,0.149274,0.126967,0.161992,0.186730,0.192660,0.138456,...,0.171940,0.163813,0.167352,0.139932,0.150779,0.152649,0.166633,0.225884,0.121954,0.118466
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.223342,0.218465,0.216847,0.117413,0.304595,0.139685,0.069260,0.100944,0.282323,...,0.180719,0.241471,0.167837,0.081130,0.300007,0.116337,0.091454,0.223127,0.185098,0.107700
50%,0.000000,0.302381,0.308759,0.293345,0.172895,0.390358,0.224679,0.144213,0.166501,0.369697,...,0.250445,0.356876,0.235320,0.123206,0.397081,0.179110,0.181070,0.343402,0.247782,0.163977
75%,1.000000,0.416442,0.408860,0.416765,0.271135,0.475490,0.340531,0.306232,0.366004,0.453030,...,0.386339,0.471748,0.373475,0.220901,0.494156,0.302520,0.305831,0.554639,0.318155,0.242949
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [22]:
# Separating the input and output from the dataset
# X = np.array(clean_n.iloc[:, :]) # Predictors 
Y = np.array(df_y['diagnosis']) # Target


In [23]:
X_train, X_test, y_train, y_test = train_test_split(clean_n, Y,
                                                    test_size = 0.2, random_state = 0)

X_train.shape
X_test.shape


(114, 31)

In [24]:
#Model Building
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

rf_default = RandomForestClassifier(random_state=42)
rf_default.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [25]:
y_pred_default = rf_default.predict(X_test)

In [26]:
# Evaluate the default model
from sklearn.metrics import accuracy_score, classification_report
print("Performance of RandomForest without Hyperparameter Tuning:")
print("Accuracy:", accuracy_score(y_test, y_pred_default))
print("Classification Report:\n", classification_report(y_test, y_pred_default))

Performance of RandomForest without Hyperparameter Tuning:
Accuracy: 0.956140350877193
Classification Report:
               precision    recall  f1-score   support

      Benign       0.95      0.99      0.97        77
   Malignant       0.97      0.89      0.93        37

    accuracy                           0.96       114
   macro avg       0.96      0.94      0.95       114
weighted avg       0.96      0.96      0.96       114



In [27]:
# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200, 300],                   # Number of trees in the forest
    'max_depth': [None, 10, 20, 30, 40],                   # Maximum depth of each tree
    'min_samples_split': [2, 5, 10, 15],                   # Minimum samples required to split a node
    'min_samples_leaf': [1, 2, 4, 6],                      # Minimum samples required to be a leaf node
    'max_features': ['auto', 'sqrt', 'log2'],              # Number of features to consider for split
    'bootstrap': [True, False],                            # Use bootstrap samples
    'criterion': ['gini', 'entropy'],                      # Split criteria
    'class_weight': [None, 'balanced', 'balanced_subsample']  # Class weighting
}

In [28]:
# Set up RandomizedSearchCV with 100 random parameter combinations to test
random_search = RandomizedSearchCV(estimator=RandomForestClassifier(random_state=42), 
                                   param_distributions=param_grid, 
                                   n_iter=100,           # Number of random parameter combinations to try
                                   cv=5,                 # 5-fold cross-validation
                                   scoring='accuracy', 
                                   n_jobs=-1, 
                                   verbose=2,
                                   random_state=42)

In [29]:
# Fit RandomizedSearchCV on the training data
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
190 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
116 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\Dell\anaconda3\Lib\site-packages\sk

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'class_weight': [None, 'balanced',
                                                         'balanced_subsample'],
                                        'criterion': ['gini', 'entropy'],
                                        'max_depth': [None, 10, 20, 30, 40],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4, 6],
                                        'min_samples_split': [2, 5, 10, 15],
                                        'n_estimators': [50, 100, 200, 300]},
                   random_state=42, scoring='accuracy', verbose=2)

In [30]:
# Print the best parameters and best score
print("Best Parameters from RandomizedSearchCV:", random_search.best_params_)
print("Best Cross-Validated Accuracy:", random_search.best_score_)

Best Parameters from RandomizedSearchCV: {'n_estimators': 50, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 20, 'criterion': 'entropy', 'class_weight': 'balanced_subsample', 'bootstrap': False}
Best Cross-Validated Accuracy: 0.9626373626373625


In [31]:
#Evaluate the tuned model
best_rf = random_search.best_estimator_
y_pred_tuned = best_rf.predict(X_test)


In [32]:
print("\nPerformance of RandomForest with Hyperparameter Tuning:")
print("Accuracy:", accuracy_score(y_test, y_pred_tuned))
print("Classification Report:\n", classification_report(y_test, y_pred_tuned))


Performance of RandomForest with Hyperparameter Tuning:
Accuracy: 0.9649122807017544
Classification Report:
               precision    recall  f1-score   support

      Benign       0.95      1.00      0.97        77
   Malignant       1.00      0.89      0.94        37

    accuracy                           0.96       114
   macro avg       0.98      0.95      0.96       114
weighted avg       0.97      0.96      0.96       114

